In [1]:
import pandas as pd
df1 = pd.read_csv("../data_gpt_labeler/labeled_datasets/final_data_labeled_1.csv")
df2 = pd.read_csv("../data_gpt_labeler/labeled_datasets/final_data_labeled_2.csv")
df3 = pd.read_csv("../data_gpt_labeler/labeled_datasets/final_data_labeled_3.csv")
df6 = pd.read_csv("../data_gpt_labeler/labeled_datasets/final_data_labeled_6.csv")
df = pd.concat([df1, df2, df3, df6])
df.shape

(40000, 8)

In [2]:
# Filtering data
filtered_df = df[df['policy_label'].notna()].reset_index(drop=True).copy()
filtered_df = filtered_df.drop(columns="Unnamed: 0")
filtered_df.to_csv("../final_lda_train_data.csv", index=False)

### Feature A

In [7]:
from modules.policy_A_module import policy_A_feature_generation
feature_A = policy_A_feature_generation(filtered_df)

[nltk_data] Downloading package wordnet to /Users/yumin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yumin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Feature B

In [8]:
from modules.policy_B_module import calculate_specificity_score
feature_B = calculate_specificity_score(filtered_df, n_workers=4)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yumin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/yumin/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/yumin/nltk_data...
[nltk_data]   Package words is already up-to-date!


### Feature D2

In [9]:
from modules.policy_D2_module import calculate_interpretability_scores_for_df
feature_D2 = calculate_interpretability_scores_for_df(filtered_df, n_workers=4)

/Users/yumin/Documents/GitHub/TikTok-TechJam-2025/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Feature E

In [11]:
from modules.policy_E_module import compute_consistency_scores
feature_E = compute_consistency_scores(filtered_df, max_workers=2)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassifi

### Feature G

In [31]:
from modules.policy_G_module import compute_policy_g_series_processed



feature_G = compute_policy_g_series_processed(filtered_df)

In [32]:
print(feature_G)

0       0.990305
1       0.999477
2       0.999410
3       0.977368
4       0.999571
          ...   
9914    0.999582
9915    0.999487
9916    0.957214
9917    0.999015
9918    0.999614
Name: policy_G_S_toxicity, Length: 9919, dtype: float64


### Concat Data into New CSV

In [33]:
df_featured_6 = pd.read_csv('../feature_engineering_model/featured_datasets/final_data_featured_6.csv')
print(df_featured_6)


      Unnamed: 0         A         B        D2         E
0              0  0.755270  0.553070  0.500000  0.750000
1              1  0.007409  0.559989  0.500000  0.986859
2              2  0.007982  0.552950  0.624615  0.645319
3              3  0.005442  0.574159  0.645191  0.707052
4              4  0.011812  0.569315  0.500000  0.989223
...          ...       ...       ...       ...       ...
9914        9914  0.008216  0.549692  0.500000  0.773126
9915        9915  0.484086  0.553054  0.576117  0.942918
9916        9916  0.007779  0.544649  0.500000  0.979812
9917        9917  0.010173  0.536573  0.500000  0.796173
9918        9918  0.557961  0.530914  0.500000  0.984082

[9919 rows x 5 columns]


In [34]:
df_temp = pd.concat([df_featured_6, feature_G], axis = 1)
print(df_temp)

      Unnamed: 0         A         B        D2         E  policy_G_S_toxicity
0              0  0.755270  0.553070  0.500000  0.750000             0.990305
1              1  0.007409  0.559989  0.500000  0.986859             0.999477
2              2  0.007982  0.552950  0.624615  0.645319             0.999410
3              3  0.005442  0.574159  0.645191  0.707052             0.977368
4              4  0.011812  0.569315  0.500000  0.989223             0.999571
...          ...       ...       ...       ...       ...                  ...
9914        9914  0.008216  0.549692  0.500000  0.773126             0.999582
9915        9915  0.484086  0.553054  0.576117  0.942918             0.999487
9916        9916  0.007779  0.544649  0.500000  0.979812             0.957214
9917        9917  0.010173  0.536573  0.500000  0.796173             0.999015
9918        9918  0.557961  0.530914  0.500000  0.984082             0.999614

[9919 rows x 6 columns]


In [35]:
df_temp = df_temp.rename(columns={"policy_G_S_toxicity":"G"})
print(df_temp)

      Unnamed: 0         A         B        D2         E         G
0              0  0.755270  0.553070  0.500000  0.750000  0.990305
1              1  0.007409  0.559989  0.500000  0.986859  0.999477
2              2  0.007982  0.552950  0.624615  0.645319  0.999410
3              3  0.005442  0.574159  0.645191  0.707052  0.977368
4              4  0.011812  0.569315  0.500000  0.989223  0.999571
...          ...       ...       ...       ...       ...       ...
9914        9914  0.008216  0.549692  0.500000  0.773126  0.999582
9915        9915  0.484086  0.553054  0.576117  0.942918  0.999487
9916        9916  0.007779  0.544649  0.500000  0.979812  0.957214
9917        9917  0.010173  0.536573  0.500000  0.796173  0.999015
9918        9918  0.557961  0.530914  0.500000  0.984082  0.999614

[9919 rows x 6 columns]


In [36]:
df_temp.to_csv("featured_datasets/final_data_featured_6.csv") #change file name so it does not overwrite

In [38]:
final_df = pd.read_csv("../feature_engineering_model/featured_datasets/final_data_featured_6.csv")
print(final_df)

      Unnamed: 0.1  Unnamed: 0         A         B        D2         E  \
0                0           0  0.755270  0.553070  0.500000  0.750000   
1                1           1  0.007409  0.559989  0.500000  0.986859   
2                2           2  0.007982  0.552950  0.624615  0.645319   
3                3           3  0.005442  0.574159  0.645191  0.707052   
4                4           4  0.011812  0.569315  0.500000  0.989223   
...            ...         ...       ...       ...       ...       ...   
9914          9914        9914  0.008216  0.549692  0.500000  0.773126   
9915          9915        9915  0.484086  0.553054  0.576117  0.942918   
9916          9916        9916  0.007779  0.544649  0.500000  0.979812   
9917          9917        9917  0.010173  0.536573  0.500000  0.796173   
9918          9918        9918  0.557961  0.530914  0.500000  0.984082   

             G  
0     0.990305  
1     0.999477  
2     0.999410  
3     0.977368  
4     0.999571  
...      

In [12]:
print(len(feature_A), len(feature_B), len(feature_D2), len(feature_E))
pd.concat([feature_A, feature_B, feature_D2, feature_E], axis=1).shape

5972 5972 5972 5972


(5972, 4)

In [ ]:
df_save = pd.DataFrame()
# df = pd.concat([feature_A, feature_B, feature_D2, feature_E, feature_G], axis=1)
df_save = pd.concat([feature_A, feature_B, feature_D2, feature_E], axis=1)
# df = df.rename(columns={0:"A", 1:"B", 2: "D2", 3:"E", 4:"G"})
df_save = df_save.rename(columns={0:"A", 1:"B", 2: "D2", 3:"E"})
df_save.to_csv("featured_datasets/final_data_featured_2.csv") #change file name so it does not overwrite

In [11]:
df_temp = pd.read_csv('../final_data_featured_filtered.csv')
df_temp["A"] = feature_A
df_temp

,A,B,D2,E,G,policy_label
0,0.151367,0.581653,0.660104,0.993421,0.999407,1.0
1,0.014161,0.565305,0.620909,0.360918,0.997275,1.0
2,0.009880,0.575038,0.551840,0.987322,0.999049,1.0
3,0.143038,0.575837,0.630236,0.859828,0.999422,1.0
4,0.384205,0.572793,0.792418,0.901759,0.999557,1.0
...,...,...,...,...,...,...
31115,0.007734,0.549692,0.500000,0.773126,0.999582,1.0
31116,0.078986,0.553054,0.576117,0.942918,0.999487,1.0
31117,0.009356,0.544649,0.500000,0.979812,0.957214,1.0
31118,0.012299,0.536573,0.500000,0.796173,0.999015,1.0


In [13]:
df_temp.to_csv("../final_data_featured_filtered.csv", index=False) #change file name so it does not overwrite

In [14]:
pd.read_csv("../final_data_featured_filtered.csv")

,A,B,D2,E,G,policy_label
0,0.151367,0.581653,0.660104,0.993421,0.999407,1.0
1,0.014161,0.565305,0.620909,0.360918,0.997275,1.0
2,0.009880,0.575038,0.551840,0.987322,0.999049,1.0
3,0.143038,0.575837,0.630236,0.859828,0.999422,1.0
4,0.384205,0.572793,0.792418,0.901759,0.999557,1.0
...,...,...,...,...,...,...
31115,0.007734,0.549692,0.500000,0.773126,0.999582,1.0
31116,0.078986,0.553054,0.576117,0.942918,0.999487,1.0
31117,0.009356,0.544649,0.500000,0.979812,0.957214,1.0
31118,0.012299,0.536573,0.500000,0.796173,0.999015,1.0
